In [83]:
# libraries
import datetime
import time
import json
from pymongo import MongoClient
from faker import Faker

# Data generation

In [84]:
class Lab2models:
    def data_generator(self, num_companies, people_per_company):
        # Connect to MongoDB - Note: added my local host
        client = MongoClient('mongodb://localhost:27017/')
        db = client['lab2']
        # delete collection data if exists from all fo the 3 models
        db.drop_collection("m1_people")
        db.drop_collection("m1_companies")
        db.drop_collection("m2_people")
        db.drop_collection("m3_companies")
        # create and obtain collection
        m1_people = db.create_collection('m1_people')
        m1_companies = db.create_collection('m1_companies')
        m2_people = db.create_collection('m2_people')
        m3_companies = db.create_collection('m3_companies')
        # It will generate data in spanish: 'es_ES'
        fake = Faker(['es_ES'])
        # Generate company data
        company_data = []
        for i in range(num_companies):
            company_name = fake.company()
            domain = fake.domain_name()
            company = {
                "domain": domain,
                "email": f"info@{domain}",
                "name": company_name,
                "url": f"www.{domain}",
                "vatNumber": fake.bothify(text='??######')
            }
            company_data.append(company)
            print(f"Company {i+1}: {company_name}")
        
        # Insert companies for Model 1
        company_ids = m1_companies.insert_many(company_data).inserted_ids

        # Generate person data
        person_data = []
        for i in range(num_companies * people_per_company):
            company_idx = i % num_companies  # Distribute people among companies
            
            first_name = fake.first_name()
            last_name = fake.last_name()
            full_name = f"{first_name} {last_name}"
            birth_date = fake.date_of_birth(minimum_age=20, maximum_age=65)
            birth_datetime = datetime.datetime.combine(birth_date, datetime.time())
            age = (datetime.datetime.now().date() - birth_date).days // 365
            
            person = {
                "age": age,
                "companyEmail": f"{first_name.lower()}@{company_data[company_idx]['domain']}",
                "dateOfBirth": birth_datetime,
                "email": fake.email(),
                "firstName": first_name,
                "fullName": full_name,
                "sex": fake.random_element(elements=('M', 'F'))
            }
            person_data.append((person, company_idx))
            print(f"Person {i+1}: {full_name}")

            # MODEL 1: Insert persons with references to companies
        print("\nInserting data for Model 1 (Referential)...")
        for i, (person, company_idx) in enumerate(person_data):
            m1_person = person.copy()
            m1_person["worksIn"] = company_ids[company_idx]  # Reference to company
            m1_people.insert_one(m1_person)
            print(f"M1 Document {i+1} inserted")
        
        # MODEL 2: Insert persons with embedded companies
        print("\nInserting data for Model 2 (Companies embedded in Persons)...")
        for i, (person, company_idx) in enumerate(person_data):
            m2_person = person.copy()
            m2_person["worksIn"] = company_data[company_idx]  # Embed company document
            m2_people.insert_one(m2_person)
            print(f"M2 Document {i+1} inserted")
        
        # MODEL 3: Insert companies with embedded persons
        print("\nInserting data for Model 3 (Persons embedded in Companies)...")
        # Group persons by company
        persons_by_company = {}
        for person, company_idx in person_data:
            if company_idx not in persons_by_company:
                persons_by_company[company_idx] = []
            persons_by_company[company_idx].append(person.copy())
        
        # Create company documents with embedded persons
        for i, company in enumerate(company_data):
            m3_company = company.copy()
            m3_company["staff"] = persons_by_company.get(i, [])
            m3_companies.insert_one(m3_company)
            print(f"M3 Document {i+1} inserted with {len(persons_by_company.get(i, []))} staff members")
        
        # Example of measuring query time
        print("\nMeasuring query time example:")
        start_time = time.time()
        result = m1_people.find_one()
        query_time = time.time() - start_time
        print(f"--- {query_time} seconds ---\nSample document: {result}")
        
        client.close()
        
        return num_companies, people_per_company
    def close_connection(self):
        if hasattr(self, 'client'):
            self.client.close()
            print("MongoDB connection closed")

In [86]:
if __name__ == "__main__":
    lab = Lab2models()

    num_companies = 30       
    people_per_company = 100  
    
    print(f"Generating data for {num_companies} companies with approximately {people_per_company} people per company...")
    print(f"Total expected documents: {num_companies} companies and {num_companies * people_per_company} people")
    
    # Medir el tiempo de generación de datos
    start_time = time.time()
    
    # Generar datos para todos los modelos
    lab.data_generator(num_companies, people_per_company)
    
    generation_time = time.time() - start_time
    
    print(f"\nData generation completed in {generation_time:.2f} seconds")
    

Generating data for 30 companies with approximately 100 people per company...
Total expected documents: 30 companies and 3000 people
Company 1: Inmobiliaria PYKY S.Coop.
Company 2: Banco Morillo S.L.N.E
Company 3: Hermanos Pavón S.C.P
Company 4: Alimentación Pol S.Coop.
Company 5: Instalaciones Carbonell y asociados S.A.
Company 6: Nogueira y Jove S.A.
Company 7: Hermanos Isern S.L.N.E
Company 8: Compañía Torrecilla & Asociados S.Com.
Company 9: Compañía IE S.L.N.E
Company 10: María Jesús Paredes Casals S.L.N.E
Company 11: Compañía Ibérica S.L.
Company 12: Rufina Posada Pombo S.L.
Company 13: Distribuciones Integrales S.Coop.
Company 14: Manufacturas Martí S.C.P
Company 15: Servicios HA S.L.N.E
Company 16: Distribuciones Vidal S.Coop.
Company 17: Hnos Sanabria S.L.
Company 18: Sebastián & Asociados S.A.
Company 19: Servicios Cerdán S.Com.
Company 20: Estrella Dalmau Francisco S.C.P
Company 21: Sebastián & Asociados S.L.
Company 22: Adalberto Benítez Caro S.L.N.E
Company 23: Desarrollo 

# Queries